#  إيمان موسى البلخي 

Prepare libraries and data

In [163]:
import pandas as pd
# here put every import you need e.g. import nltk
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer
import tashaphyne.arabic_const as arabconst
import tashaphyne.arabic_const as arabconst
import nltk
from nltk import word_tokenize
from nltk.tokenize import *
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.collocations import *
import matplotlib.pyplot as plt           
import matplotlib
from wordcloud import WordCloud
import arabic_reshaper
from bidi.algorithm import get_display
from nltk.stem.lancaster import LancasterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from numpy import *
import array

Prapere The Comparison Dictionary

In [164]:
model_comparison_table = {}

model_comparison_table['question_step_number'] = []
model_comparison_table['model_name'] = []
model_comparison_table['parameters'] = []
model_comparison_table['preprocessing_methods'] = []
model_comparison_table['accuracy'] = []


# Question [1]: Load the data

شرح ما يقوم به الكود

In [165]:
# Naming uniformity
def Naming_uniformity_label(lebel_tweet):
    
    lebel_tweet = lebel_tweet.lower()
    lebel_tweet = re.sub(r'\s', '', lebel_tweet)
    lebel_tweet = re.sub(r'ir\w+t', 'irrelevent', lebel_tweet)
    lebel_tweet = re.sub(r'n\w+l', 'neutral', lebel_tweet)
    lebel_tweet = re.sub(r'n\w+e', 'negative', lebel_tweet)
    lebel_tweet = re.sub(r'p\w+e', 'positive', lebel_tweet)
    return lebel_tweet

def Naming_uniformity_all_label(dataframe):
    dataframe['label'] = dataframe['label'].apply(lambda x: "".join(preprocess_label(str(x))))
    return dataframe


In [168]:
# dataframe train
dataframe_train = pd.read_csv('train.csv')
print("Total_tweets_train:                                    ",dataframe_train.shape[0] )
dataframe_train.dropna(inplace = True)
print("\nTotal_tweets after deleting lines containing NAN:",dataframe_train.shape[0])
dataframe=dataframe_train.drop_duplicates(subset=['text','label'],keep='first')
dataframe_train=dataframe
print("\nTotal_tweets after deleting duplicate tweets:    ",dataframe_train.shape[0])


dataframe_train=Naming_uniformity_all_label(dataframe_train)
print('************')
print(dataframe_train['label'].value_counts())
print('************')


dataframe_train_x=dataframe_train['text']
dataframe_train_y=dataframe_train['label']

# negative:0,positive:1,neutral:2,irrelevent:3
for i in dataframe_train_y.index:
    if dataframe_train_y[i]=='negative':
         dataframe_train_y[i]=0
    elif dataframe_train_y[i]=='positive':
        dataframe_train_y[i]=1
    elif dataframe_train_y[i]=='neutral':
        dataframe_train_y[i]=2
    elif dataframe_train_y[i]=='irrelevent':
        dataframe_train_y[i]=3

        
# Only for positive and negative tweets
dataframe_train_pn=dataframe_train[dataframe_train['label']<2]


dataframe_train_pn_x=dataframe_train_pn['text']
dataframe_train_pn_y=dataframe_train_pn['label']
  
print("number tweets positive+negative: ",dataframe_train_pn.shape)

dataframe_train_x=dataframe_train['text']
dataframe_train_y=dataframe_train['label']

Total_tweets_train:                                     33924

Total_tweets after deleting lines containing NAN: 33633

Total_tweets after deleting duplicate tweets:     26254
************
neutral       11559
positive       5713
negative       5472
irrelevent     3510
Name: label, dtype: int64
************
number tweets positive+negative:  (11185, 2)


In [169]:
# dataframe vaild
dataframe_vaild = pd.read_csv('valid.csv')
print("Total_tweets_vaild:                                    ",dataframe_vaild.shape[0] )
dataframe_vaild.dropna(inplace = True)
print("\nTotal_tweets after deleting lines containing NAN:",dataframe_vaild.shape[0])
dataframe=dataframe_vaild.drop_duplicates(subset=['text','label'],keep='first')
dataframe_vaild=dataframe
print("\nTotal_tweets after deleting duplicate tweets:    ",dataframe_vaild.shape[0])

# Naming uniformity
dataframe_train=Naming_uniformity_all_label(dataframe_vaild)
print("*********")
print(dataframe_vaild['label'].value_counts())
print("**********")

dataframe_vaild_x=dataframe_vaild['text']
dataframe_vaild_y=dataframe_vaild['label']

# negative:0,positive:1,neutral:2,irrelevent:3
for i in dataframe_vaild_y.index:
    if dataframe_vaild_y[i]=='negative':
         dataframe_vaild_y[i]=0
    elif dataframe_vaild_y[i]=='positive':
        dataframe_vaild_y[i]=1
    elif dataframe_vaild_y[i]=='neutral':
        dataframe_vaild_y[i]=2
    elif dataframe_vaild_y[i]=='irrelevent':
        dataframe_vaild_y[i]=3

# Only for positive and negative tweets
dataframe_vaild_pn=dataframe_vaild[dataframe_vaild['label']<2]

dataframe_vaild_pn_x=dataframe_vaild_pn['text']
dataframe_vaild_pn_y=dataframe_vaild_pn['label']

print("number tweets (positive+negative): ",dataframe_vaild_pn.shape)

Total_tweets_vaild:                                     7269

Total_tweets after deleting lines containing NAN: 7213

Total_tweets after deleting duplicate tweets:     6815
*********
neutral       2980
positive      1518
negative      1403
irrelevent     914
Name: label, dtype: int64
**********
number tweets (positive+negative):  (2921, 2)


In [170]:
# datafram test
dataframe_test = pd.read_csv('test.csv')
print("Total_tweets_test:                                    ",dataframe_test.shape[0] )
dataframe_test.dropna(inplace = True)
print("\nTotal_tweets after deleting lines containing NAN:",dataframe_test.shape[0])
dataframe=dataframe_test.drop_duplicates(subset=['text','label'],keep='first')
dataframe_test=dataframe
print("\nTotal_tweets after deleting duplicate tweets:    ",dataframe_test.shape[0])

# Naming uniformity
dataframe_test=Naming_uniformity_all_label(dataframe_test)
print("*********")
print(dataframe_test['label'].value_counts())
print("**********")

dataframe_test_x=dataframe_test['text']
dataframe_test_y=dataframe_test['label']

# negative:0,positive:1,neutral:2,irrelevent:3
for i in dataframe_test_y.index:
    if dataframe_test_y[i]=='negative':
         dataframe_test_y[i]=0
    elif dataframe_test_y[i]=='positive':
        dataframe_test_y[i]=1
    elif dataframe_test_y[i]=='neutral':
        dataframe_test_y[i]=2
    elif dataframe_test_y[i]=='irrelevent':
        dataframe_test_y[i]=3

# Only for positive and negative tweets
dataframe_test_pn=dataframe_test[dataframe_test['label']<2]

dataframe_test_pn_x=dataframe_test_pn['text']
dataframe_test_pn_y=dataframe_test_pn['label']

print("number tweets positive+negative: ",dataframe_test_pn.shape)

dataframe_test_pn_x=dataframe_test['text']
dataframe_test_pn_y=dataframe_test['label']

Total_tweets_test:                                     7270

Total_tweets after deleting lines containing NAN: 7204

Total_tweets after deleting duplicate tweets:     6783
*********
neutral       2906
positive      1510
negative      1437
irrelevent     930
Name: label, dtype: int64
**********
number tweets positive+negative:  (2947, 2)


In [171]:
print("Total_tweets_train : ",dataframe_train_pn.shape[0])
print("Total_tweets_vaild : ",dataframe_vaild_pn.shape[0])
print("Total_tweets_test  : ",dataframe_test_pn.shape[0] )

Total_tweets_train :  11185
Total_tweets_vaild :  2921
Total_tweets_test  :  2947


# Question [2]: Baseline: Bag of Words with Logistic Regression

شرح ما يقوم به الكود

In [172]:
# train && vaild

def BOW(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

X_train_counts, count_vectorizer = BOW(dataframe_train_pn_x)
X_vaild_counts = count_vectorizer.transform(dataframe_vaild_pn_x)

In [173]:
clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [174]:
from sklearn.metrics import accuracy_score
dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8065730914070524

In [175]:
model_comparison_table['question_step_number'].append("2")
model_comparison_table['model_name'].append("baseline logistic regression with bag of words")
model_comparison_table['parameters'].append("default")
model_comparison_table['preprocessing_methods'].append("none")
model_comparison_table['accuracy'].append(accuracy)
model_comparison_table

{'question_step_number': ['2'],
 'model_name': ['baseline logistic regression with bag of words'],
 'parameters': ['default'],
 'preprocessing_methods': ['none'],
 'accuracy': [0.8065730914070524]}

# Question [3]: Text Cleaning and Normalization on Bag of Words with Logistic Regression

Prapere The Comparison Dictionary for this step

In [176]:
bag_logistic_comparison_table = {}

bag_logistic_comparison_table['question_step_number'] = []
bag_logistic_comparison_table['preprocessing_methods'] = []
bag_logistic_comparison_table['accuracy'] = []

## [3.1] 
شرح ما يقوم به الكود


In [177]:
def delete_links(text ): 
    text = re.sub(r'https?\/\/S+', '',  text) # remove the hyperlink
    text = re.sub(r'http\S+', '',  text) # remove the hyperlink
    text = re.sub(r'www\S+', '',  text) # remove the www
    return text

def delete_mentions(text): 
    return re.sub(r'@[A-Za-z0-9]+', '', text)

def delete_hashtags1(text):
    return re.sub(r"#(\S+)",'', text)

def delete_hashtags2(text):
    return re.sub(r"#",'', text)

def Delete_duplicate_characters(text):
    return re.sub(r'(.)\1+', r'\1', text)

def replace_Hindi_numbers_to_Arabic_number(text):
    text=re.sub(r'٠', r'0', text)
    text=re.sub(r'١', r'1', text)
    text=re.sub(r'٢', r'2', text)
    text=re.sub(r'٣', r'3', text)
    text=re.sub(r'٤', r'4', text)
    text=re.sub(r'٥', r'5', text)
    text=re.sub(r'٦', r'6', text)
    text=re.sub(r'٧', r'7', text)
    text=re.sub(r'٨', r'8', text)
    text=re.sub(r'٩', r'9', text)
    return text 

def standardize_number(text):
    return re.sub(r'\d+','*',text)

def delete_all_number(text):
    return re.sub(r'\d+','',text)
def delete_punctuation_marks(text): 
    text = re.sub(r'\n|[!\"\＄%&\'\(\)\*\+,-\./:;<=>\?\[\\\]\^`{\|}~،“؟”]^#(\S+)', ' ', text)
    return text



def delete_imojis(text): 
    imoji_regex=re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = re.sub(imoji_regex ,' ', text);
    return text

def delete_NON_Arabics(text): 
    Regex_non_arabic=r'\d+|[\u0660-\u0669]|[\!\"\＄%&\'\(\)\*\+,-\./:;<=>\?\[\\\]\^`{\|}~،“؟”]|#|_|[\u0600-\u06FF]|\s|[u"\U0001F600-\U0001F64F""\U0001F300-\U0001F5FF"u"\U0001F680-\U0001F6FF"u"\U0001F1E0-\U0001F1FF"u"\U00002702-\U000027B0"u"\U000024C2-\U0001F251"]+'
    arabic_words = "".join(re.findall(Regex_non_arabic, text))
    return arabic_words

def stop_word(text):
    text = [w for w in text if w not in stopwords.words("arabic")]


def Stemming_English(text_token):
    lancaster = nltk.LancasterStemmer() 
    return [(t,lancaster.stem(t)) for t in text_token]

def Stemming_Arabic (text):
    text=nltk.word_tokenize(text)
    ArListem = ArabicLightStemmer()    
    return [(t,ArListem.light_stem(t)) for t in text]

def normalize_hamza(text):
    arabconst.ALEFAT_PAT.sub(arabconst.ALEF, text)
    return arabconst.HAMZAT_PAT.sub(arabconst.HAMZA, text)

def normalize_ALEFAT(text):
    return arabconst.ALEFAT_PAT.sub(arabconst.ALEF, text)
def strip_tatweel(text):
    return re.sub(u'[%s]' % arabconst.TATWEEL, '', text)
def strip_tashkeel(text):
    return arabconst.HARAKAT_PAT.sub('', text)
def replace_multiple_spaces_with_a_single_space(text):
    return re.sub(' +', ' ', text)
def delete_Frequency__tweet(list_tweet):
    return set(list_tweet)

def preprocess(text,
               delete_link ,
               delete_mention , 
               delete_hashtag1 ,
               delete_hashtag2,
               Delete_duplicate_character,
               standardize_num,
               delete_all_num,
               delete_punctuation_mark,
               delete_imoji,
               delete_nonArabic,
               stop__word,
               Stemming_Englis,
               Stemming_Arabi,
               normalize_hamz,
               normalize_ALEFA,
               strip_tatwee,
               strip_tashkee,
               replace_multiple_spaces_with_a_single_spac,
               delete_Frequency__twee ):
    
    if delete_link==1:
        text=delete_links(text)
        
    if delete_mention==1:
        text=delete_mentions(text)
        
    if delete_hashtag1==1:
        text=delete_hashtags1(text)
        
    if delete_hashtag2==1:
        text=delete_hashtags2(text)
        
    if Delete_duplicate_character==1:
        text=Delete_duplicate_characters(text)
        
    if standardize_num==1:
        text=standardize_number(text)
        
    if delete_all_num==1:
        text=delete_all_number(text)
        
    if delete_punctuation_mark==1:
        text=delete_punctuation_marks(text)
        
    if delete_imoji==1:
        text=delete_imojis(text)
        
    if delete_nonArabic==1:
        text=delete_NON_Arabics(text)
        
    if stop__word==1:
        text=stop_word(text)
        
    if Stemming_Englis==1:
        text=Stemming_English(text)
        
    if Stemming_Arabi==1:
        text=Stemming_Arabic(text)
        
    if normalize_hamz==1:
        text=normalize_hamza(text)
        
    if normalize_ALEFA==1:
        text=normalize_ALEFAT(text)
        
    if strip_tatwee==1:
        text=strip_tatweel(text)
        
    if strip_tashkee==1:
        text=strip_tashkeel(text)
        
    if replace_multiple_spaces_with_a_single_spac==1:
        text=replace_multiple_spaces_with_a_single_space(text)
        
    if delete_Frequency__twee==1:
        text=delete_Frequency__tweet(text)   

    return text


In [178]:
trainX_delete_link_delete_mention  = [preprocess(t ,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_link_delete_mention  = [preprocess(t ,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

In [179]:
X_train_counts, count_vectorizer = BOW(trainX_delete_link_delete_mention)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_link_delete_mention)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [180]:
# test your model
dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8052036973639165

إزالة الروابط والمنشن ادى الى خفض الدقة  

In [181]:
bag_logistic_comparison_table['question_step_number'].append("3.1")
bag_logistic_comparison_table['preprocessing_methods'].append("delete link +delete_mention")
bag_logistic_comparison_table['accuracy'].append(accuracy)


## [3.2] 

In [182]:
trainX_delete_hashtags1 = [preprocess(t ,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_hashtags1 = [preprocess(t ,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

In [183]:
X_train_counts, count_vectorizer = BOW(trainX_delete_hashtags1)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_hashtags1)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [184]:
# test your model
dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8000684697021568

هنالك تابعين لازالة الهاشتاك  
الاول يحذف الكلمات المرفقة مع الهاشتاك 
وعند تطبيق هذا التابع ادى الى خفض الدقة 

In [185]:
bag_logistic_comparison_table['question_step_number'].append("3.2.1")
bag_logistic_comparison_table['preprocessing_methods'].append("remove hashtags1")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1', '3.2.1'],
 'preprocessing_methods': ['delete link +delete_mention', 'remove hashtags1'],
 'accuracy': [0.8052036973639165, 0.8000684697021568]}

In [186]:
trainX_delete_hashtags2 = [preprocess(t ,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_hashtags2 = [preprocess(t ,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

In [187]:
X_train_counts, count_vectorizer = BOW(trainX_delete_hashtags2)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_hashtags2)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [188]:
dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8069154399178363

ازالة الهاشتاغ عن طريق التابع الثاني الذي يحذف فقظ اشارة #

ادى هذا التابع الى رفع الدقة 

In [189]:
bag_logistic_comparison_table['question_step_number'].append("3.2.2")
bag_logistic_comparison_table['preprocessing_methods'].append("remove hashtags2")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1', '3.2.1', '3.2.2'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2'],
 'accuracy': [0.8052036973639165, 0.8000684697021568, 0.8069154399178363]}

## [3.3] 

In [190]:
trainX_delete_Delete_duplicate_characters= [preprocess(t ,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_Delete_duplicate_characters = [preprocess(t ,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

In [191]:
X_train_counts, count_vectorizer = BOW(trainX_delete_Delete_duplicate_characters)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_Delete_duplicate_characters)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [192]:
dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8017802122560767

In [193]:
bag_logistic_comparison_table['question_step_number'].append("3.3")
bag_logistic_comparison_table['preprocessing_methods'].append("Delete duplicate characters")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1', '3.2.1', '3.2.2', '3.3'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767]}

ادى حذف المحارف المكررة من النص الى خفض الدقة  

## [3.4] 

### [3.4.1] 

In [194]:
trainX_replace_Hindi_numbers_to_Arabic_number = [preprocess(t ,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_replace_Hindi_numbers_to_Arabic_number = [preprocess(t ,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_replace_Hindi_numbers_to_Arabic_number)
X_vaild_counts = count_vectorizer.transform(vaildX_replace_Hindi_numbers_to_Arabic_number)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8113659705580281

تحويل الارقام الهندية لارقام عربية ادى الى رفع الدقة 

In [195]:
bag_logistic_comparison_table['question_step_number'].append("3.4.1")
bag_logistic_comparison_table['preprocessing_methods'].append("replace Hindi numbers to Arabic number")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table


{'question_step_number': ['3.1', '3.2.1', '3.2.2', '3.3', '3.4.1'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281]}

### [3.4.2] 

In [196]:
trainX_standardize_number = [preprocess(t ,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_standardize_number = [preprocess(t ,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_standardize_number)
X_vaild_counts = count_vectorizer.transform(vaildX_standardize_number)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8096542280041081

In [197]:
bag_logistic_comparison_table['question_step_number'].append("3.4.2")
bag_logistic_comparison_table['preprocessing_methods'].append("standardize number")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1', '3.2.1', '3.2.2', '3.3', '3.4.1', '3.4.2'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081]}

توحيد كتابة الارقام واستبدالها بنجمة ادى الى رفع الدقة ولكن تحويل الارقام الهندية الى ارقام عربية رفع الدقة اكثر

مما يوضح أهمية الارقام في التغريديات 

### [3.4.3] 

In [198]:
trainX_delete_all_number = [preprocess(t ,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_all_number = [preprocess(t ,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_delete_all_number)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_all_number)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8065730914070524

In [199]:
bag_logistic_comparison_table['question_step_number'].append("3.4.3")
bag_logistic_comparison_table['preprocessing_methods'].append("delete all number")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524]}

حذف الارقام كلياا من التغريدات كان اسوء حالة 

من بين الحالات الثلاثة السابقة 

## [3.5] 

### [3.5.1] 

In [200]:
trainX_delete_NON_Arabics = [preprocess(t ,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_NON_Arabics = [preprocess(t ,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_delete_NON_Arabics)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_NON_Arabics)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8079424854501883

In [201]:
bag_logistic_comparison_table['question_step_number'].append("3.5.1")
bag_logistic_comparison_table['preprocessing_methods'].append("delete NON Arabics")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table


{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883]}

حذف المحارف غير العربية ادى الى رفع الدقة 

### [3.5.2] 

In [202]:

trainX_delete_punctuation_marks = [preprocess(t ,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_punctuation_marks = [preprocess(t ,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_delete_punctuation_marks)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_punctuation_marks)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8065730914070524

In [203]:
bag_logistic_comparison_table['question_step_number'].append("3.5.2")
bag_logistic_comparison_table['preprocessing_methods'].append("delete punctuation marks ")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks '],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524]}

حذف علامات الترقيم جميعها لم يؤثر على الدقة  

### [3.5.3] 

In [204]:

trainX_delete_imojis = [preprocess(t ,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_delete_imojis = [preprocess(t ,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_delete_imojis)
X_vaild_counts = count_vectorizer.transform(vaildX_delete_imojis)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8069154399178363

In [205]:
bag_logistic_comparison_table['question_step_number'].append("3.5.3")
bag_logistic_comparison_table['preprocessing_methods'].append("delete imojis ")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table


{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis '],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363]}

حذف الوجوه التعبيرية ادى لرفع الدقة  

## [3.6] 

In [206]:
# trainX_stop__word = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
# vaildX_stop__word = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

# X_train_counts, count_vectorizer = BOW(trainX_stop__word)
# X_vaild_counts = count_vectorizer.transform(vaildX_stop__word)

# clf = LogisticRegression(random_state=40)
# dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
# clf.fit(X_train_counts, dataframe_train_pn_y)

# y_predicted = clf.predict(X_vaild_counts)

# dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
# accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
# accuracy

In [207]:
accuracy=0.8060230914070721

In [208]:
bag_logistic_comparison_table['question_step_number'].append("3.6")
bag_logistic_comparison_table['preprocessing_methods'].append("stop word ")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word '],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721]}

حذف كلمات التوقف ادى لخفض الدقة

تم تخزين الدقة في المتحول السابق وتعليق الكود لان تنفيذ هالتابع كتير بطيئ وعم ياخذ وقت كتير بسب انو هو عم بشيك على كل تغريدة على حدى وبقارنها مع كل كلمات التوقف 

## [3.7] 

In [209]:

# trainX_Stemming_Arabic = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0) for t in dataframe_train_pn_x ]
# vaildX_Stemming_Arabic = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

# X_train_counts, count_vectorizer = BOW(trainX_Stemming_Arabic)
# X_vaild_counts = count_vectorizer.transform(vaildX_Stemming_Arabic)

# clf = LogisticRegression(random_state=40)
# dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
# clf.fit(X_train_counts, dataframe_train_pn_y)

# y_predicted = clf.predict(X_vaild_counts)

# dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
# accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
# accuracy

In [210]:
accuracy=0

In [211]:
bag_logistic_comparison_table['question_step_number'].append("3.7")
bag_logistic_comparison_table['preprocessing_methods'].append("Stemming Arabic")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0]}

## [3.8] 

### [3.8.1] 

In [212]:

trainX_normalize_hamz = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_normalize_hamz = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_normalize_hamz)
X_vaild_counts = count_vectorizer.transform(vaildX_normalize_hamz)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy             

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8058883943854844

In [213]:
bag_logistic_comparison_table['question_step_number'].append("3.8.1")
bag_logistic_comparison_table['preprocessing_methods'].append("normalize hamz")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7',
  '3.8.1'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic',
  'normalize hamz'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0,
  0.8058883943854844]}

توحيد كتابة الهمزات ادى لخفض الدقة 

### [3.8.2] 

In [214]:

trainX_normalize_ALEFA = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0) for t in dataframe_train_pn_x ]
vaildX_normalize_ALEFA = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_normalize_ALEFA)
X_vaild_counts = count_vectorizer.transform(vaildX_normalize_ALEFA)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8034919548099966

In [215]:
bag_logistic_comparison_table['question_step_number'].append("3.8.2")
bag_logistic_comparison_table['preprocessing_methods'].append("normalize ALEFA")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7',
  '3.8.1',
  '3.8.2'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic',
  'normalize hamz',
  'normalize ALEFA'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0,
  0.8058883943854844,
  0.8034919548099966]}

توحيد الألفات ادى لخفض الدقة 

### [3.8.3] 

In [216]:

trainX_strip_tatwee = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0) for t in dataframe_train_pn_x ]
vaildX_strip_tatwee = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_strip_tatwee)
X_vaild_counts = count_vectorizer.transform(vaildX_strip_tatwee)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8052036973639165

In [217]:
bag_logistic_comparison_table['question_step_number'].append("3.8.3")
bag_logistic_comparison_table['preprocessing_methods'].append("strip tatwee")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7',
  '3.8.1',
  '3.8.2',
  '3.8.3'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic',
  'normalize hamz',
  'normalize ALEFA',
  'strip tatwee'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0,
  0.8058883943854844,
  0.8034919548099966,
  0.8052036973639165]}

حذف التطويل ادى لخفض الدقة 

### [3.8.4] 

In [218]:
trainX_strip_tashkee = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0) for t in dataframe_train_pn_x ]
vaildX_strip_tashkee = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_strip_tatwee)
X_vaild_counts = count_vectorizer.transform(vaildX_strip_tatwee)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8052036973639165

In [219]:
bag_logistic_comparison_table['question_step_number'].append("3.8.4")
bag_logistic_comparison_table['preprocessing_methods'].append("strip tashkee")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7',
  '3.8.1',
  '3.8.2',
  '3.8.3',
  '3.8.4'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic',
  'normalize hamz',
  'normalize ALEFA',
  'strip tatwee',
  'strip tashkee'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0,
  0.8058883943854844,
  0.8034919548099966,
  0.8052036973639165,
  0.8052036973639165]}

حذف علامات التشكيل ادى لخفض الدقة 

## [3.9] 

In [220]:
trainX_replace_multiple_spaces_with_a_single_spac = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0) for t in dataframe_train_pn_x ]
vaildX_replace_multiple_spaces_with_a_single_spac = [preprocess(t ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0) for t in dataframe_vaild_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_replace_multiple_spaces_with_a_single_spac)
X_vaild_counts = count_vectorizer.transform(vaildX_replace_multiple_spaces_with_a_single_spac)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8065730914070524

In [221]:
bag_logistic_comparison_table['question_step_number'].append("3.9")
bag_logistic_comparison_table['preprocessing_methods'].append("replace multiple spaces with a single spac")
bag_logistic_comparison_table['accuracy'].append(accuracy)
bag_logistic_comparison_table

{'question_step_number': ['3.1',
  '3.2.1',
  '3.2.2',
  '3.3',
  '3.4.1',
  '3.4.2',
  '3.4.3',
  '3.5.1',
  '3.5.2',
  '3.5.3',
  '3.6',
  '3.7',
  '3.8.1',
  '3.8.2',
  '3.8.3',
  '3.8.4',
  '3.9'],
 'preprocessing_methods': ['delete link +delete_mention',
  'remove hashtags1',
  'remove hashtags2',
  'Delete duplicate characters',
  'replace Hindi numbers to Arabic number',
  'standardize number',
  'delete all number',
  'delete NON Arabics',
  'delete punctuation marks ',
  'delete imojis ',
  'stop word ',
  'Stemming Arabic',
  'normalize hamz',
  'normalize ALEFA',
  'strip tatwee',
  'strip tashkee',
  'replace multiple spaces with a single spac'],
 'accuracy': [0.8052036973639165,
  0.8000684697021568,
  0.8069154399178363,
  0.8017802122560767,
  0.8113659705580281,
  0.8096542280041081,
  0.8065730914070524,
  0.8079424854501883,
  0.8065730914070524,
  0.8069154399178363,
  0.8060230914070721,
  0,
  0.8058883943854844,
  0.8034919548099966,
  0.8052036973639165,
  0.8052

 حذف الفراغات المتكررة واستبدالها بفراغ واحد لم يؤثر على الدقة 

## [3.10]

التوابع الاضافية كانت تابع الهاشتغ الذي يقوم بحذف فقط اشارة الهاشتغ وتم تجريبة في الطلب الثاني 

وادى لرفع الدقة 

## Print Table

In [222]:
df = pd.DataFrame(bag_logistic_comparison_table)
df

,question_step_number,preprocessing_methods,accuracy
0,3.1,delete link +delete_mention,0.805204
1,3.2.1,remove hashtags1,0.800068
2,3.2.2,remove hashtags2,0.806915
3,3.3,Delete duplicate characters,0.801780
4,3.4.1,replace Hindi numbers to Arabic number,0.811366
5,3.4.2,standardize number,0.809654
6,3.4.3,delete all number,0.806573
7,3.5.1,delete NON Arabics,0.807942
8,3.5.2,delete punctuation marks,0.806573
9,3.5.3,delete imojis,0.806915


## Your best model
ما هي العمليات الأفضل؟ اكتبها هنا

ازالة الهاشتغ (حذف فقط الاشارة)

تحويل الارقام الهندية للارقام العربية وتبين ان هنالك اهمية لوجود الارقام في التغريدات

حذف المحارف غير العربية

حذف الوجوه التعبيرية 


In [223]:
# train && test
trainX_best = [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_train_pn_x ]
testX_best= [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_test_pn_x ]

X_train_counts, count_vectorizer = BOW(trainX_best)
X_vaild_counts = count_vectorizer.transform(testX_best)

clf = LogisticRegression(random_state=40)
dataframe_train_pn_y=dataframe_train_pn_y.astype('int') 
clf.fit(X_train_counts, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_test_pn_y=dataframe_test_pn_y.astype('int')
accuracy=accuracy_score(dataframe_test_pn_y, y_predicted)
accuracy 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.3489606368863335

In [224]:
model_comparison_table['question_step_number'].append("3")
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['parameters'].append("blah")
model_comparison_table['preprocessing_methods'].append("remove hashtags2 + replace Hindi numbers to Arabic number+ delete NON Arabics+ delete imojis")
model_comparison_table['accuracy'].append(accuracy)
model_comparison_table

{'question_step_number': ['2', '3'],
 'model_name': ['baseline logistic regression with bag of words',
  'logistic regression with bag of words'],
 'parameters': ['default', 'blah'],
 'preprocessing_methods': ['none',
  'remove hashtags2 + replace Hindi numbers to Arabic number+ delete NON Arabics+ delete imojis'],
 'accuracy': [0.8065730914070524, 0.3489606368863335]}

# Question [4]: TF-IDF with Logistic Regression

Prapere The Comparison Dictionary for this step

In [225]:
tf_idf_comparison_table = {}

tf_idf_comparison_table['question_step_number'] = []
tf_idf_comparison_table['model_name'] = []
tf_idf_comparison_table['parameters'] = []
tf_idf_comparison_table['accuracy'] = []

## [4.1] 
شرح ما يقوم به الكود


In [226]:
trainX_best = [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_train_pn_x ]
vaildX_best= [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_vaild_pn_x ]

In [227]:
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.6, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

In [228]:
# test your model
accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7418692228688806

In [229]:
tf_idf_comparison_table['question_step_number'].append("4.1")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("default parameters")
tf_idf_comparison_table['accuracy'].append(accuracy)


## [4.2] 

In [230]:
# change ngram_range
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.6, ngram_range=(1, 1))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7292023279698733

In [231]:
tf_idf_comparison_table['question_step_number'].append("4.2.1.2")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("ngram_range(1,1)")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [232]:
# change ngram_range
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.6, ngram_range=(1, 3))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7415268743580965

In [233]:
tf_idf_comparison_table['question_step_number'].append("4.2.1.3")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("ngram_range(1,3)")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [234]:
# change ngram_range
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7422115713796645

In [235]:
tf_idf_comparison_table['question_step_number'].append("4.2.1.4")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("ngram_range(1,4)")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [236]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=7, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7528243752139678

بهذه القيمة (1,4) حصلنا على دقة جيد افضل من باقي القيم التي تم تجريبها لنثبتها ونجرب برمتر اخر 

In [237]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.1")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 7")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [238]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=6, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7562478603218076

In [239]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.2")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 6")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [240]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7586442998972954

In [241]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.3")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 5")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [242]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=4, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7613830879835672

In [243]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.4")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 4")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [244]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=3, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7723382403286546

In [245]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.5")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 3")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [246]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7850051352276618

In [247]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.6")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 2")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [248]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8028072577884287

In [249]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.7")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 1")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [250]:
# change min_df && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=0, max_df=0.6, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8028072577884287

In [251]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.2.8")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("min_df : 0")
tf_idf_comparison_table['accuracy'].append(accuracy)


بعد عدة تجريبات ارتفعت الدقة عند القيمة 0  لل min

In [252]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=0, max_df=0.5, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8045190003423485

In [253]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.3.1")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("max_df : 0.5")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [254]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=0, max_df=0.4, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.7993837726805888

In [255]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.3.2")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("max_df : 0.4")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [256]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=0, max_df=1, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.74871619308456

In [257]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.3.3")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("max_df : 1")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [258]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=0, max_df=0.7, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8028072577884287

In [259]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.3.4")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("max_df : 0.7")
tf_idf_comparison_table['accuracy'].append(accuracy)


In [260]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_pn_y, y_predicted)
accuracy

0.8045190003423485

In [261]:
# change min_df
tf_idf_comparison_table['question_step_number'].append("4.2.3.5")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append("max_df : 0.55")
tf_idf_comparison_table['accuracy'].append(accuracy)


من خلال التجريب وجدنا ان البرمترات التي رفعت الدقة كانت

ngram=1,4    and        min=1         and          max=0.5

وكانت الدقة 0.8045190003423485 

In [262]:
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('clf',RidgeClassifier()),
# ])
# parameters = {
#     'tfidf__max_df': (0.25, 0.5, 0.75),
#     'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
#     'clf__estimator__alpha': (1e-2, 1e-3)
# }

# grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=2, verbose=3)
# grid_search_tune.fit(Xtrain, Ytrain)

# print("Best parameters set:")
# print(grid_search_tune.best_estimator_.steps)

بعد الانتهاء من عمليات التجريب ..تم الانتبهاء الى الطريقة السابقة التى تجرب كل القيم مع كل الاحتمالات وترجع الاحتمالات الافضل مع الدقة الافضل 

In [263]:
trainX_best = [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_train_pn_x ]
testX_best= [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_test_pn_x ]

In [264]:
# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_test_tfidf = tfidf_vectorizer.transform(testX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_pn_y)

y_predicted = clf.predict(X_test_tfidf)

accuracy=accuracy_score(dataframe_test_pn_y, y_predicted)
accuracy

0.35294117647058826

In [265]:
tf_idf_comparison_table['question_step_number'].append("4.2")
tf_idf_comparison_table['model_name'].append("logistic regression with TF_IDF")
tf_idf_comparison_table['parameters'].append(" ngram:(1,4)+ min: 1 + max : 0.55")
tf_idf_comparison_table['accuracy'].append(accuracy)

## [4.3] 

In [266]:
# train && vaild
from sklearn.svm import SVC 

# change max_df  && Installations min_df =0  && Installations ngram_range(1,4)
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

classifier = SVC(kernel='linear', random_state=0)  
classifier.fit(X_train_tfidf, dataframe_train_pn_y)  

#Predicting the test set result  
y_pred= classifier.predict(X_vaild_tfidf) 
accuracy=accuracy_score(dataframe_vaild_pn_y, y_pred)
accuracy

0.8141047586442999

In [267]:
tf_idf_comparison_table['question_step_number'].append("4.3.1")
tf_idf_comparison_table['model_name'].append("svm with TF_IDF")
tf_idf_comparison_table['parameters'].append(" ngram:(1,4)+ min: 1 + max : 0.55")
tf_idf_comparison_table['accuracy'].append(accuracy)

In [268]:
# train &&test 
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_test_tfidf = tfidf_vectorizer.transform(testX_best)

classifier = SVC(kernel='linear', random_state=0)  
classifier.fit(X_train_tfidf, dataframe_train_pn_y)  

#Predicting the test set result  
y_pred= classifier.predict(X_test_tfidf) 
accuracy=accuracy_score(dataframe_test_pn_y, y_pred)
accuracy

0.36016511867905054

In [269]:
tf_idf_comparison_table['question_step_number'].append("4.3.2")
tf_idf_comparison_table['model_name'].append("svm with TF_IDF")
tf_idf_comparison_table['parameters'].append(" ngram:(1,4)+ min: 1 + max : 0.55")
tf_idf_comparison_table['accuracy'].append(accuracy)

## Print Table

In [270]:
df = pd.DataFrame(tf_idf_comparison_table)
df

,question_step_number,model_name,parameters,accuracy
0,4.1,logistic regression with TF_IDF,default parameters,0.741869
1,4.2.1.2,logistic regression with TF_IDF,"ngram_range(1,1)",0.729202
2,4.2.1.3,logistic regression with TF_IDF,"ngram_range(1,3)",0.741527
3,4.2.1.4,logistic regression with TF_IDF,"ngram_range(1,4)",0.742212
4,4.2.2.1,logistic regression with TF_IDF,min_df : 7,0.752824
5,4.2.2.2,logistic regression with TF_IDF,min_df : 6,0.756248
6,4.2.2.3,logistic regression with TF_IDF,min_df : 5,0.758644
7,4.2.2.4,logistic regression with TF_IDF,min_df : 4,0.761383
8,4.2.2.5,logistic regression with TF_IDF,min_df : 3,0.772338
9,4.2.2.6,logistic regression with TF_IDF,min_df : 2,0.785005


## Your best model
ما هو النموذج الأفضل؟ اكتبه هنا

بما ان دقة نموذج svm اعلى

قد يكون هو النموذج الافضل 

# Question [5]: Deep NN

Prapere The Comparison Dictionary for this step

In [725]:
dnn_comparison_table = {}

dnn_comparison_table['question_step_number'] = []
dnn_comparison_table['model_name'] = []
dnn_comparison_table['parameters'] = []
dnn_comparison_table['accuracy'] = []

## [5.1] 
شرح ما يقوم به الكود


In [10]:
pip install –-upgrade pip

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '–-upgrade'


In [2]:
pip install –-user –upgrade pip

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '–-user'


In [3]:
!pip install tensorflow
# !pip install keras 

  Using cached tensorflow-2.11.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.11.0-cp38-cp38-win_amd64.whl (266.3 MB)
  Using cached protobuf-3.19.6-cp38-cp38-win_amd64.whl (896 kB)
  Using cached tensorflow_io_gcs_filesystem-0.29.0-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached flatbuffers-22.12.6-py2.py3-none-any.whl (26 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached grpcio-1.51.1-cp38-cp38-win_amd64.whl (3.7 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Using cached google_auth-2.15.0-py2.py3-no

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\importlib_metadata-3.10.0.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:


In [9]:
import tensorflow.compat.v2 as tf
from keras import backend
from keras.models import Sequential
from keras import layers

input_dim = len(trainX_best)  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.summary()

history = model.fit(trainX_best, dataframe_train_pn_y,
                     epochs=100,
                     verbose=False,
                     validation_split = 0.2,
                     batch_size=10)

loss, accuracy = model.evaluate(trainX_best, dataframe_train_pn_y, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(testX_best, dataframe_test_pn_y, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

ModuleNotFoundError: No module named 'tensorflow'

لم استطع تدريب الشبكة العصبونية العميقة لان مكتبة الtesorflow غير موجودة عندي ولم استطع تثبيتها


In [ ]:
dnn_comparison_table['question_step_number'].append("5.1")
dnn_comparison_table['model_name'].append("Fully Connected DNN")
dnn_comparison_table['parameters'].append("balh + blah + .... + blah")
dnn_comparison_table['accuracy'].append(accuracy)

## [5.2] 

### Your best model

In [ ]:
# test the model
accuracy = 0

In [ ]:
model_comparison_table['question_step_number'].append("5.2")
model_comparison_table['model_name'].append("Fully Connected DNN")
model_comparison_table['parameters'].append("balh blah blah")
model_comparison_table['preprocessing_methods'].append("balh + blah + .... + blah")
model_comparison_table['accuracy'].append(accuracy)

## [5.3] 

## [5.4] 

### Your best model

In [ ]:
# test the model
accuracy = 0

In [ ]:
model_comparison_table['question_step_number'].append("5.4")
model_comparison_table['model_name'].append("CNN")
model_comparison_table['parameters'].append("balh blah blah")
model_comparison_table['preprocessing_methods'].append("balh + blah + .... + blah")
model_comparison_table['accuracy'].append(accuracy)

## [5.5] 

## Print Table

In [ ]:
import pandas as pd

df = pd.DataFrame(dnn_comparison_table)
df

,question_step_number,model_name,parameters,accuracy
0,5.1,Fully Connected DNN,balh + blah + .... + blah,0


# Question [6]: Train on all Classes

## [6.1] 
شرح ما يقوم به الكود


In [125]:
trainX_best = [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_train_x ]
vaildX_best= [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_vaild_x ]
testX_best= [preprocess(t ,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0) for t in dataframe_test_x ]

هون المفروض اعمل resampling  للداتا بس معاد ضل وقت 

In [ ]:
# bow &&LogisticRegression     all data
X_train_counts, count_vectorizer = BOW(trainX_best)
X_vaild_counts = count_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
dataframe_train_y=dataframe_train_pn_y.astype('int')

clf.fit(X_train_counts, dataframe_train_y)

y_predicted = clf.predict(X_vaild_counts)

dataframe_vaild_pn_y=dataframe_vaild_pn_y.astype('int')
accuracy=accuracy_score(dataframe_vaild_y, y_predicted)
accuracy 

In [135]:
# tfidf &&LogisticRegression     all data
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, dataframe_train_y)

y_predicted = clf.predict(X_vaild_tfidf)

accuracy=accuracy_score(dataframe_vaild_y, y_predicted)
accuracy

In [ ]:
# tfidf &&svm     all data
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.55, ngram_range=(1,4))
X_train_tfidf = tfidf_vectorizer.fit_transform(trainX_best)
X_vaild_tfidf = tfidf_vectorizer.transform(vaildX_best)

classifier = SVC(kernel='linear', random_state=0)  
classifier.fit(X_train_tfidf, dataframe_train_y)  

#Predicting the test set result  
y_pred= classifier.predict(X_vaild_tfidf) 
accuracy=accuracy_score(dataframe_vaild_y, y_pred)
accuracy

In [ ]:
# test your model
accuracy = 0

In [136]:
model_comparison_table['question_step_number'].append("6.1")
model_comparison_table['model_name'].append("blah with all classes")
model_comparison_table['parameters'].append("balh blah blah")
model_comparison_table['preprocessing_methods'].append("balh + blah + .... + blah")
model_comparison_table['accuracy'].append(accuracy)

## [6.2] 


# Final Results

In [137]:
df = pd.DataFrame(model_comparison_table)
df

,question_step_number,model_name,parameters,preprocessing_methods,accuracy
0,2,baseline logistic regression with bag of words,default,none,0.806573
1,3,logistic regression with bag of words,blah,remove hashtags2 + replace Hindi numbers to Ar...,0.803190
2,6.1,blah with all classes,balh blah blah,balh + blah + .... + blah,0.828979


In [271]:
df.to_csv("[ايمان موسى البلخي].csv", index=False)